In [ ]:
!pip install tpot

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.4/87.4 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 14.4 MB/s eta 0:00:00
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11939 sha256=bd0c4f757fb415f6fb395f61a0227db92430bb26b7a31dee362fdade7705c677
  Stored in directory: /root/.cache/pip/wheels/af/f9/87/bf5b3d565c2a007b4dae9d8142dccc85a9f164e517062dd519
Successfully built stopit


In [ ]:
#from tpot import  TPOTClassifier
from sklearn.model_selection import train_test_split
from __future__ import print_function
import sys,tempfile, urllib, os
import pandas as pd
import numpy as np

In [ ]:
#dataset preprocessing & train test split <specially for churn>
# Load the dataset
df = pd.read_csv('/content/transport_survey.csv')
new_col = [
    "sex", "age", "category", "district", "uni", "freq", "mode",
    "gotime", "rettime", "taka", "road", "network", "cost", "sign",
    "design", "smooth", "safety", "behavior", "speed", "weather",
    "noise", "dust", "emergency", "rules", "jam", "security",
    "suggestion", "sentiment"
]
df.columns = new_col
df.head()

,sex,age,category,district,uni,freq,mode,gotime,rettime,taka,...,speed,weather,noise,dust,emergency,rules,jam,security,suggestion,sentiment
0,Male,16-25,Student,Dhaka,MIST,Daily,"Rickshaw, On Foot",0-20 mins,20-40 mins,100-150,...,2,5,1,1,1,3,2,3,-,No
1,Male,16-25,Student,Dhaka,MIST,Daily,Rickshaw,0-20 mins,0-20 mins,50-100,...,3,3,3,3,4,3,2,2,By improving the application of traffic facili...,Yes
2,Male,16-25,Student,Chattogram,CU,Daily,CNG,80-100 mins,80-100 mins,100-150,...,4,4,4,2,1,2,2,1,It would be Helpful for us if the vehicles had...,No
3,Male,26-40,Teacher,Sylhet,SUST,Daily,"Rickshaw, Auto-Rickshaw, Public Bus",60-80 mins,60-80 mins,150-200,...,3,2,2,2,4,2,2,2,Would've been great if the traffic polices app...,No
4,Female,16-25,Student,Dhaka,BUET,Daily,"University Bus, Public Bus, Metro Rail",20-40 mins,60-80 mins,50-100,...,3,2,1,1,1,1,2,2,NaN,Yes


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Define the categorical columns
categorical_columns = [
    "road", "network", "cost", "sign",
    "design", "smooth", "safety", "behavior",
    "speed", "weather", "noise", "dust",
    "emergency", "rules", "jam", "security",
    "sentiment"
]

# Create a copy of the original DataFrame to work with
churn_df_trans = df.copy()

# Apply LabelEncoder to the 'sentiment' column
churn_df_trans['sentiment'] = df['sentiment'].replace({
    'Yes': 2,
    'No': 1
})

# Now you can update the original DataFrame with the transformed values
df.update(churn_df_trans)

# Display the first few rows of the updated DataFrame
df.head()


<ipython-input-38-2afcc2e74ed6>:17: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  churn_df_trans['sentiment'] = df['sentiment'].replace({


,sex,age,category,district,uni,freq,mode,gotime,rettime,taka,...,speed,weather,noise,dust,emergency,rules,jam,security,suggestion,sentiment
0,Male,16-25,Student,Dhaka,MIST,Daily,"Rickshaw, On Foot",0-20 mins,20-40 mins,100-150,...,2,5,1,1,1,3,2,3,-,1
1,Male,16-25,Student,Dhaka,MIST,Daily,Rickshaw,0-20 mins,0-20 mins,50-100,...,3,3,3,3,4,3,2,2,By improving the application of traffic facili...,2
2,Male,16-25,Student,Chattogram,CU,Daily,CNG,80-100 mins,80-100 mins,100-150,...,4,4,4,2,1,2,2,1,It would be Helpful for us if the vehicles had...,1
3,Male,26-40,Teacher,Sylhet,SUST,Daily,"Rickshaw, Auto-Rickshaw, Public Bus",60-80 mins,60-80 mins,150-200,...,3,2,2,2,4,2,2,2,Would've been great if the traffic polices app...,1
4,Female,16-25,Student,Dhaka,BUET,Daily,"University Bus, Public Bus, Metro Rail",20-40 mins,60-80 mins,50-100,...,3,2,1,1,1,1,2,2,NaN,2


In [ ]:
df_X = df.drop(["sex", "age", "category", "district", "uni", "freq", "mode",
                "gotime", "rettime", "taka",
                "suggestion", "sentiment"], axis=1)
#churn_df_X = churn_df_X.drop("customerID", axis=1)
df_y = df['sentiment']


X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, train_size=0.8, test_size=0.2)
# Convert y_train and y_test to integers explicitly
y_train = y_train.astype(int)
y_test = y_test.astype(int)

import multiprocessing

if __name__ == '__main__':
    multiprocessing.set_start_method('forkserver', force=True)
    tpot =  TPOTClassifier(generations=5, population_size=20, verbosity=2,n_jobs = 20, random_state=50)
    tpot.fit(X_train, y_train)


print(tpot.score(X_test, y_test))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1230: FutureWarning: passing a class to None is deprecated and will be removed in 1.8. Use an instance of the class instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1270: FutureWarning: passing a class to None is deprecated and will be removed in 1.8. Use an instance of the class instead.
  warnings.warn(


is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_classifier
is_classifier
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_classifier
is_classifier
is_classifier
is_classifier
is_classifier
is_classifier
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier
is_classifier
is_classifier
is_regressor
is_classifier
is_regressor
is_classifier


Optimization Progress:   0%|          | 0/120 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.7977272727272727

Generation 2 - Current best internal CV score: 0.8068181818181819

Generation 3 - Current best internal CV score: 0.8068181818181819

Generation 4 - Current best internal CV score: 0.8068181818181819

Generation 5 - Current best internal CV score: 0.8068181818181819

Best pipeline: KNeighborsClassifier(input_matrix, n_neighbors=89, p=1, weights=distance)
0.8090909090909091


In [ ]:
print(tpot.score(X_test, y_test))

0.8090909090909091


In [ ]:
tpot.export('tpot_transport_pipeline.py')

In [7]:
!pip install pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.6/169.6 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.1/486.1 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.8/21.8 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 111.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.7/80.7 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 87.6 MB/s eta 0:00:00
   ━━

In [1]:
from pycaret.classification import *

In [3]:
from sklearn.model_selection import train_test_split
from __future__ import print_function
import sys,tempfile, urllib, os
import pandas as pd
import numpy as np
#dataset preprocessing & train test split <specially for churn>
# Load the dataset
df = pd.read_csv('/content/transport_survey.csv')
new_col = [
    "sex", "age", "category", "district", "uni", "freq", "mode",
    "gotime", "rettime", "taka", "road", "network", "cost", "sign",
    "design", "smooth", "safety", "behavior", "speed", "weather",
    "noise", "dust", "emergency", "rules", "jam", "security",
    "suggestion", "sentiment"
]
df.columns = new_col
#df.head()

# Apply LabelEncoder to the 'sentiment' column
df['sentiment'] = df['sentiment'].replace({
    'Yes': 2,
    'No': 1
})

#df.head()



,sex,age,category,district,uni,freq,mode,gotime,rettime,taka,...,speed,weather,noise,dust,emergency,rules,jam,security,suggestion,sentiment
0,Male,16-25,Student,Dhaka,MIST,Daily,"Rickshaw, On Foot",0-20 mins,20-40 mins,100-150,...,2,5,1,1,1,3,2,3,-,1
1,Male,16-25,Student,Dhaka,MIST,Daily,Rickshaw,0-20 mins,0-20 mins,50-100,...,3,3,3,3,4,3,2,2,By improving the application of traffic facili...,2
2,Male,16-25,Student,Chattogram,CU,Daily,CNG,80-100 mins,80-100 mins,100-150,...,4,4,4,2,1,2,2,1,It would be Helpful for us if the vehicles had...,1
3,Male,26-40,Teacher,Sylhet,SUST,Daily,"Rickshaw, Auto-Rickshaw, Public Bus",60-80 mins,60-80 mins,150-200,...,3,2,2,2,4,2,2,2,Would've been great if the traffic polices app...,1
4,Female,16-25,Student,Dhaka,BUET,Daily,"University Bus, Public Bus, Metro Rail",20-40 mins,60-80 mins,50-100,...,3,2,1,1,1,1,2,2,NaN,2


In [4]:
df_setup = setup(data = df, target = 'sentiment', session_id=3435, ignore_features=["sex", "age", "category", "district", "uni", "freq", "mode",
    "gotime", "rettime", "taka", "suggestion"])

,Description,Value
0,Session id,3435
1,Target,sentiment
2,Target type,Binary
3,Target mapping,"1: 0, 2: 1"
4,Original data shape,"(550, 28)"
5,Transformed data shape,"(550, 17)"
6,Transformed train set shape,"(385, 17)"
7,Transformed test set shape,"(165, 17)"
8,Ignore features,11
9,Numeric features,16


In [6]:
df_setup

In [7]:
compare_models(fold=5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
nb,Naive Bayes,0.8130,0.8557,0.8130,0.8251,0.8159,0.5932,0.5985,0.0280
lr,Logistic Regression,0.7948,0.8386,0.7948,0.7909,0.7887,0.5133,0.5200,1.2740
gbc,Gradient Boosting Classifier,0.7896,0.8433,0.7896,0.7855,0.7858,0.5088,0.5116,0.1280
lda,Linear Discriminant Analysis,0.7844,0.8323,0.7844,0.7812,0.7782,0.4894,0.4969,0.0280
lightgbm,Light Gradient Boosting Machine,0.7844,0.8382,0.7844,0.7824,0.7817,0.5020,0.5047,0.2140
ridge,Ridge Classifier,0.7818,0.8326,0.7818,0.7774,0.7735,0.4772,0.4866,0.0280
rf,Random Forest Classifier,0.7818,0.8507,0.7818,0.7784,0.7771,0.4885,0.4933,0.2020
knn,K Neighbors Classifier,0.7792,0.8297,0.7792,0.7787,0.7776,0.4952,0.4973,0.0400
et,Extra Trees Classifier,0.7662,0.8412,0.7662,0.7617,0.7619,0.4541,0.4573,0.1720
xgboost,Extreme Gradient Boosting,0.7636,0.8281,0.7636,0.7615,0.7610,0.4553,0.4577,0.1260


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

GaussianNB(priors=None, var_smoothing=1e-09)